<a href="https://colab.research.google.com/github/samer-glitch/TADP-Enhancing-Trustworthiness-in-Federated-Learning-Through-Provenance-Driven-Governance/blob/main/Adult_%2B_many_state_of_the_art_comparators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ADULT DATASET WITH 3 RUNS confidence interval => MEAN AND STD**


In [1]:
# ============================================================
#  FEDERATED LEARNING — ADULT INCOME DATASET
#  Robust FL + Governance: FedAvg, FedProx, Median, Krum,
#  Power-of-Choice, Static PScore, Dynamic PScore, PScore-weighted
# ============================================================

import os
import time
import math
import random
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    roc_auc_score,
)

import torch
import torch.nn as nn
import torch.optim as optim

# ============================================================
#  GLOBAL CONFIG
# ============================================================

RANDOM_STATE = 42
N_CLIENTS = 10
N_ROUNDS = 15
ALPHA_DIRICHLET = 0.5

BATCH_SIZE = 64
LOCAL_EPOCHS = 4
LR = 1e-3

DEVICE_POWER_KW = 0.1
CO2_PER_KWH = 0.4
MODEL_SIZE_MB = 1.0

# Robust FL / selection parameters
POC_TOP_K = 3              # Power-of-Choice
KRUM_F_BYZANTINE = 1       # Assumed number of Byzantine clients for Krum

# Static PScore threshold percentile (Gate 1)
GOVERNANCE_PERCENTILE = 30.0  # 30th percentile for basic gate

# Dynamic PScore-v3 hyperparams (soft down-weighting)
DYN_WARMUP_ROUNDS = 5
DYN_ALPHA_PROV = 0.7      # weight for governance vs sample-size
DYN_LOW_PENALTY = 0.5     # down-weight for low-provenance clients (less harsh than 0.3)

# Dynamic gate hyperparams (hard gate after warm-up)
GATE_WARMUP_ROUNDS = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
#  UTILS
# ============================================================

def set_global_seed(seed: int = RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def estimate_energy_kwh(t_s: float) -> float:
    return DEVICE_POWER_KW * (t_s / 3600.0)


def estimate_co2_kg(kwh: float) -> float:
    return kwh * CO2_PER_KWH

# ============================================================
#  DATA LOADING + PREPROCESSING
# ============================================================

def load_adult_dataset(csv_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    print("📊 Loading & preprocessing Adult dataset...")

    df = pd.read_csv(csv_path, header=0, na_values=["?"])
    expected_cols = 15

    if df.shape[1] == expected_cols:
        df.columns = [
            "age", "workclass", "fnlwgt", "education", "education-num",
            "marital-status", "occupation", "relationship", "race", "sex",
            "capital-gain", "capital-loss", "hours-per-week",
            "native-country", "income"
        ]

    df = df.dropna().reset_index(drop=True)

    # Normalize income labels
    df["income"] = (
        df["income"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(" ", "", regex=False)
    )

    # Remove header-like garbage rows if any
    mask_bad = df["income"].isin(["income", "income."])
    if mask_bad.any():
        print(f"   ⚠️ Removing {mask_bad.sum()} header-like rows")
        df = df[~mask_bad].reset_index(drop=True)

    mapping = {
        "<=50k": 0,
        ">50k": 1,
        "<=50k,": 0,
        ">50k,": 1,
    }
    y = df["income"].map(mapping)

    if y.isna().any():
        df["income"] = (
            df["income"]
            .str.replace("K", "k", regex=False)
            .str.replace(",", "", regex=False)
        )
        y = df["income"].map(mapping)
        if y.isna().any():
            print("   ⚠️ Fallback mapping on weird labels")
            y = df["income"].apply(lambda v: 0 if "<=50" in v else 1)

    y = y.astype(int).values

    num_cols = ["age", "fnlwgt", "education-num",
                "capital-gain", "capital-loss", "hours-per-week"]
    cat_cols = [c for c in df.columns if c not in num_cols + ["income"]]

    scaler = StandardScaler()
    X_num = scaler.fit_transform(df[num_cols].values)

    enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    X_cat = enc.fit_transform(df[cat_cols])

    X = np.hstack([X_num, X_cat]).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
    )

    print(f"   Train: {len(y_train)}, Test: {len(y_test)}")
    print(f"   Features: {X.shape[1]}\n")
    return X_train, X_test, y_train, y_test

# ============================================================
#  NON-IID DIRICHLET CLIENT SPLITTER
# ============================================================

def create_dirichlet_clients(
    X: np.ndarray, y: np.ndarray, n_clients: int, alpha: float
) -> List[Dict]:

    print(f"🔧 Creating Non-IID clients (Dirichlet α={alpha})")

    n_classes = len(np.unique(y))
    idx_by_class = [np.where(y == k)[0] for k in range(n_classes)]
    client_indices = [[] for _ in range(n_clients)]

    for k in range(n_classes):
        idx_k = idx_by_class[k]
        np.random.shuffle(idx_k)

        proportions = np.random.dirichlet(alpha * np.ones(n_clients))
        proportions = (proportions * len(idx_k)).astype(int)

        diff = len(idx_k) - proportions.sum()
        for i in range(abs(diff)):
            proportions[i % n_clients] += int(np.sign(diff))

        start = 0
        for cid in range(n_clients):
            end = min(start + proportions[cid], len(idx_k))
            client_indices[cid].extend(idx_k[start:end])
            start = end

    clients = []
    for cid in range(n_clients):
        idx = np.array(client_indices[cid])
        Xc, yc = X[idx], y[idx]
        dist = dict(zip(*np.unique(yc, return_counts=True)))
        print(f"   Client {cid}: {len(yc)} samples → {dist}")
        clients.append({"id": cid, "X": Xc, "y": yc})

    print("✅ Clients created.\n")
    return clients

# ============================================================
#  MODEL
# ============================================================

class AdultNet(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)


def init_global_model(in_features: int) -> nn.Module:
    return AdultNet(in_features).to(DEVICE)


def get_model_params(model: nn.Module) -> Dict[str, torch.Tensor]:
    return {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}


def set_model_params(model: nn.Module, params: Dict[str, torch.Tensor]):
    model.load_state_dict(params, strict=True)


def average_state_dicts(
    param_list: List[Dict[str, torch.Tensor]],
    weights: List[float],
) -> Dict[str, torch.Tensor]:
    weights = np.array(weights, dtype=float)
    weights = weights / (weights.sum() + 1e-12)

    avg = {}
    for k in param_list[0].keys():
        stacked = torch.stack([
            p[k].float() * float(w) for p, w in zip(param_list, weights)
        ])
        avg[k] = stacked.sum(dim=0)
    return avg

# ============================================================
#  ROBUST AGGREGATION HELPERS (MEDIAN, KRUM)
# ============================================================

def median_aggregate(param_list: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    out = {}
    for k in param_list[0].keys():
        stacked = torch.stack([p[k].float() for p in param_list])  # [C, ...]
        median_vals = torch.median(stacked, dim=0).values
        out[k] = median_vals
    return out


def _param_distance(p1: Dict[str, torch.Tensor], p2: Dict[str, torch.Tensor]) -> float:
    dist = 0.0
    for k in p1.keys():
        diff = (p1[k].float() - p2[k].float()).view(-1)
        dist += torch.sum(diff * diff).item()
    return dist


def krum_aggregate(param_list: List[Dict[str, torch.Tensor]], f_byz: int) -> Tuple[Dict[str, torch.Tensor], int]:
    """
    Classical Krum:
      - For each client update i, compute sum of distances to its
        closest (m - f_byz - 2) other updates.
      - Choose update with minimal score.
    """
    m = len(param_list)
    if m <= 2 * f_byz + 2:
        # Not enough clients for Krum assumptions, fallback to FedAvg
        avg = average_state_dicts(param_list, [1.0] * m)
        return avg, -1

    dists = np.zeros((m, m), dtype=float)
    for i in range(m):
        for j in range(i + 1, m):
            d = _param_distance(param_list[i], param_list[j])
            dists[i, j] = dists[j, i] = d

    scores = []
    for i in range(m):
        # distances from i to others
        others = np.delete(dists[i], i)
        # sort and sum top m - f_byz - 2 closest
        nb_closest = m - f_byz - 2
        score_i = np.sum(np.partition(others, nb_closest)[:nb_closest])
        scores.append(score_i)

    winner = int(np.argmin(scores))
    return param_list[winner], winner

# ============================================================
#  LOCAL TRAINING (FEDAVG) + FEDPROX
# ============================================================

CRITERION = nn.BCEWithLogitsLoss()


def local_train(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
) -> Tuple[Dict[str, torch.Tensor], float, float]:

    """
    Standard local training used by FedAvg, Median, Krum, PoC, etc.
    """

    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    n_features = X.shape[1]
    local_model = AdultNet(n_features).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.float32)).float().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            loss = CRITERION(logits, yb)
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).cpu().numpy()

    acc = accuracy_score(y, preds)

    return get_model_params(local_model), train_time, acc


def local_train_fedprox(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
    mu: float,
) -> Tuple[Dict[str, torch.Tensor], float, float]:
    """
    Local training with FedProx proximal term:
      loss_total = BCE + (mu/2) * ||w_local - w_global||^2
    """
    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    n_features = X.shape[1]
    local_model = AdultNet(n_features).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    global_params = {k: v.detach().clone() for k, v in global_model.state_dict().items()}

    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.float32)).float().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            base_loss = CRITERION(logits, yb)

            # Proximal term
            prox_term = 0.0
            for (name, param) in local_model.named_parameters():
                prox_term = prox_term + torch.sum((param - global_params[name].to(DEVICE))**2)

            loss = base_loss + (mu / 2.0) * prox_term
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).cpu().numpy()

    acc = accuracy_score(y, preds)

    return get_model_params(local_model), train_time, acc

# ============================================================
#  GLOBAL EVALUATION
# ============================================================

def evaluate_global_model(
    model: nn.Module,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> Dict[str, float]:

    model.eval()
    with torch.no_grad():
        X_t = torch.from_numpy(X_test).float().to(DEVICE)
        logits = model(X_t)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    rec = recall_score(y_test, preds)

    try:
        auc = roc_auc_score(y_test, probs)
    except ValueError:
        auc = float("nan")

    return {"accuracy": acc, "f1": f1, "recall": rec, "auc": auc}

# ============================================================
#  PSCORE UTILITIES
# ============================================================

def get_static_pscores(n_clients: int) -> List[float]:
    # Example PScore vector from your runs
    base = [
        4.05, 3.92, 3.94, 3.03, 3.28,
        2.83, 2.39, 1.58, 2.37, 1.51
    ]
    return base[:n_clients]


def compute_governance_threshold(pscores: List[float], percentile: float) -> float:
    arr = np.array(pscores, dtype=float)
    return float(np.percentile(arr, percentile))

def minmax_normalize_pscores(pscores: List[float]) -> np.ndarray:
    ps = np.array(pscores, dtype=float)
    p_min, p_max = ps.min(), ps.max()
    p_rng = p_max - p_min if p_max > p_min else 1.0
    return (ps - p_min) / p_rng

# ============================================================
#  BASELINE FEDAVG
# ============================================================

def run_fedavg(clients, X_test, y_test, pscores):
    print("🎯 FedAvg")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedAvg",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  FEDPROX
# ============================================================

def run_fedprox(clients, X_test, y_test, pscores, mu: float = 0.01):
    print("\n🎯 FedProx (mu={})".format(mu))

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train_fedprox(global_model, c["X"], c["y"], mu)
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedProx",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  MEDIAN
# ============================================================

def run_median(clients, X_test, y_test, pscores):
    print("\n🎯 Median")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg = median_aggregate(local_params)
        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Median",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  KRUM
# ============================================================

def run_krum(clients, X_test, y_test, pscores):
    print("\n🎯 Krum")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    chosen_counts = [0] * len(clients)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg, chosen = krum_aggregate(local_params, KRUM_F_BYZANTINE)
        if chosen >= 0:
            chosen_counts[chosen] += 1
            print(f"     🟢 Krum chose client ~{chosen}")
        else:
            print("     🟢 Fallback to FedAvg (not enough clients for Krum)")

        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Krum",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": 1,
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),  # Krum doesn't use PScore
    }

# ============================================================
#  POWER-OF-CHOICE (TOP-K ACC)
# ============================================================

def run_power_of_choice(clients, X_test, y_test, pscores):
    print("\n🎯 Power-of-Choice (top-{} acc)".format(POC_TOP_K))

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    last_selected = []

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights, accs = [], [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))
            accs.append(acc)

        accs = np.array(accs)
        top_idx = np.argsort(-accs)[:POC_TOP_K]
        last_selected = top_idx.tolist()
        print(f"     Selected clients: {last_selected}")

        sel_params = [local_params[i] for i in top_idx]
        sel_weights = [weights[i] for i in top_idx]

        avg = average_state_dicts(sel_params, sel_weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * POC_TOP_K * MODEL_SIZE_MB

    return {
        "Scenario": "Power-of-Choice",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(last_selected),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in last_selected])),
    }

# ============================================================
#  PSCORE-WEIGHTED FEDAVG (ALL CLIENTS, NO GATE)
# ============================================================

def run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores):
    """
    All clients always participate.
    Aggregation weight: |D_i| * f(P_i) with monotone PScore scaling.
    """
    print("\n🎯 PScore-Weighted FedAvg (all clients, no gate)")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)  # in [0,1]
    # map to e.g. [0.5, 1.5]
    f_ps = 0.5 + 1.0 * ps_norm

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * float(f_ps[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "PScoreWeightedFedAvgAll",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


# ============================================================
#  STATIC PSCORE (HARD GATE + PSCORE-WEIGHTED FEDAVG)
# ============================================================

def run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov: float):
    print("\n🎯 Static PScore Gate (hard gate + PScore-weighted FedAvg)")
    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)

    accepted_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    print(f"   Threshold={t_gov:.3f} → accepted: {accepted_ids}")

    active = [c for c in clients if c["id"] in accepted_ids]
    total_t = 0.0

    if len(active) == 0:
        print("   ⚠️ No clients above threshold; falling back to FedAvg over all clients.")
        return run_fedavg(clients, X_test, y_test, pscores)

    ps_norm = minmax_normalize_pscores(pscores)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []

        for c in active:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * (0.5 + 1.0 * ps_norm[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(active) * MODEL_SIZE_MB

    return {
        "Scenario": "StaticPScoreGate",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(active),
        "Trained Clients": len(active),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in accepted_ids])),
    }

# ============================================================
#  DYNAMIC PSCORE-v3 (SOFT PENALTY, NO FILTERING)
# ============================================================

def run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov: float):
    """
    Warm-up rounds: vanilla FedAvg over all clients.
    After warm-up: all clients train, but aggregation weights reflect
    static PScore + governance penalty for low-provenance clients.
    """
    print("\n🎯 Dynamic PScore-v3 (warm-up + governance-weighted FedAvg)")
    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)

    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)

    high_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    low_ids = [i for i, s in enumerate(pscores) if s < t_gov]

    print(f"   Static threshold={t_gov:.3f}")
    print(f"   High-provenance clients: {high_ids}")
    print(f"   Low-provenance clients (penalized): {low_ids}")

    for r in range(1, N_ROUNDS + 1):
        mode = "warm-up" if r <= DYN_WARMUP_ROUNDS else "gov-weighted"
        print(f"  Round {r} [{mode}]")

        local_params, weights = [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            if r <= DYN_WARMUP_ROUNDS:
                eff_w = len(c["y"])
            else:
                norm_p = ps_norm[cid]
                if cid in low_ids:
                    norm_p *= DYN_LOW_PENALTY
                # Mix governance weight with sample-size
                g_factor = DYN_ALPHA_PROV * norm_p + (1.0 - DYN_ALPHA_PROV)
                eff_w = len(c["y"]) * max(g_factor, 1e-3)

            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "DynamicPScoreV3",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),       # all contribute, but reweighted
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in high_ids])),
    }

# ============================================================
#  MAIN PIPELINE WITH MULTIPLE RUNS
# ============================================================

def get_dataset_path():
    """Get dataset path - only prompt once"""
    try:
        from google.colab import files
        print("\n⬆️ Please upload adult.csv ...\n")
        uploaded = files.upload()
        if len(uploaded) == 0:
            raise ValueError("❌ No file uploaded.")
        fname = list(uploaded.keys())[0]
        print(f"Using uploaded file: {fname}")
        return fname
    except ImportError:
        if not os.path.exists("adult.csv"):
            raise FileNotFoundError(
                "❌ Not in Colab AND no local 'adult.csv' found.\n"
                "Please put adult.csv in your working directory."
            )
        print("Using local adult.csv")
        return "adult.csv"

def run_single_experiment(run_id: int, csv_path: str):
    """Run a single experiment and return results"""
    print(f"\n{'='*80}")
    print(f"🏃‍♂️ RUN {run_id + 1}/3")
    print(f"{'='*80}")

    set_global_seed(RANDOM_STATE + run_id)  # Different seed for each run

    # ----------------------------------------
    # 2) Load dataset
    # ----------------------------------------
    X_train, X_test, y_train, y_test = load_adult_dataset(csv_path)

    # ----------------------------------------
    # 3) Create Non-IID clients
    # ----------------------------------------
    clients = create_dirichlet_clients(
        X_train,
        y_train,
        n_clients=N_CLIENTS,
        alpha=ALPHA_DIRICHLET,
    )

    # ----------------------------------------
    # 4) PScore vector + governance threshold
    # ----------------------------------------
    pscores = get_static_pscores(N_CLIENTS)
    print("📊 Static PScore values:")
    for cid, s in enumerate(pscores):
        print(f"   Client {cid}: {s:.2f}")
    print()

    t_gov = compute_governance_threshold(pscores, GOVERNANCE_PERCENTILE)
    print(f"🔒 Static governance threshold from {GOVERNANCE_PERCENTILE:.0f}th percentile: {t_gov:.3f}\n")

    # ----------------------------------------
    # 5) Run baselines + governance variants
    # ----------------------------------------
    all_results = []

    # Baselines
    all_results.append(run_fedavg(clients, X_test, y_test, pscores))
    all_results.append(run_fedprox(clients, X_test, y_test, pscores, mu=0.01))
    all_results.append(run_median(clients, X_test, y_test, pscores))
    all_results.append(run_krum(clients, X_test, y_test, pscores))
    all_results.append(run_power_of_choice(clients, X_test, y_test, pscores))

    # Governance-aware variants
    all_results.append(run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores))
    all_results.append(run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov))
    all_results.append(run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov))

    return all_results


def compute_statistics(all_runs_results):
    """Compute mean ± std for all metrics across runs"""
    # Get all unique scenario names from all runs
    all_scenarios = set()
    for run_results in all_runs_results:
        for scenario_result in run_results:
            all_scenarios.add(scenario_result["Scenario"])

    # Initialize storage for metrics
    metrics = {}
    for scenario in all_scenarios:
        metrics[scenario] = {
            "Final Accuracy": [],
            "Final F1-Score": [],
            "Final Recall": [],
            "Final AUC-ROC": [],
            "Total Training Time (s)": [],
            "Total Energy (kWh)": [],
            "Total CO2 (kg)": [],
            "Total Communication (MB)": [],
            "Selected Clients": [],
            "Trained Clients": [],
            "Avg PScore Selected": [],
        }

    # Collect data from all runs
    for run_results in all_runs_results:
        for scenario_result in run_results:
            scenario_name = scenario_result["Scenario"]
            for metric in metrics[scenario_name].keys():
                if metric in scenario_result:
                    metrics[scenario_name][metric].append(scenario_result[metric])

    # Compute statistics
    stats_results = []
    for scenario_name, scenario_metrics in metrics.items():
        stats_row = {"Scenario": scenario_name}

        for metric_name, values in scenario_metrics.items():
            if values:  # Only compute if we have data
                mean_val = np.mean(values)
                std_val = np.std(values)

                if metric_name in ["Final Accuracy", "Final F1-Score", "Final Recall", "Final AUC-ROC"]:
                    # For metrics where we want mean ± std format
                    stats_row[metric_name] = f"{mean_val:.4f} ± {std_val:.4f}"
                elif metric_name in ["Total Training Time (s)", "Total Energy (kWh)", "Total CO2 (kg)"]:
                    stats_row[metric_name] = f"{mean_val:.2f} ± {std_val:.2f}"
                elif metric_name in ["Total Communication (MB)", "Selected Clients", "Trained Clients"]:
                    stats_row[metric_name] = f"{mean_val:.1f} ± {std_val:.1f}"
                elif metric_name == "Avg PScore Selected":
                    stats_row[metric_name] = f"{mean_val:.2f} ± {std_val:.2f}"
                else:
                    stats_row[metric_name] = f"{mean_val:.4f} ± {std_val:.4f}"
            else:
                stats_row[metric_name] = "N/A"

        stats_results.append(stats_row)

    return stats_results


def main():
    # Get dataset path once
    csv_path = get_dataset_path()

    # Run experiment 3 times
    all_runs_results = []

    for run_id in range(3):
        run_results = run_single_experiment(run_id, csv_path)
        all_runs_results.append(run_results)

    # Compute statistics
    stats_results = compute_statistics(all_runs_results)

    # ----------------------------------------
    # Final summary with statistics
    # ----------------------------------------
    print("\n" + "=" * 120)
    print("📊 FINAL RESULTS — Adult Dataset (Governance + Robust FL) - Mean ± Std over 3 runs")
    print("=" * 120)

    df = pd.DataFrame(stats_results)

    cols_order = [
        "Scenario",
        "Final Accuracy",
        "Final F1-Score",
        "Final Recall",
        "Final AUC-ROC",
        "Total Training Time (s)",
        "Total Energy (kWh)",
        "Total CO2 (kg)",
        "Total Communication (MB)",
        "Selected Clients",
        "Trained Clients",
        "Avg PScore Selected",
    ]
    cols_order = [c for c in cols_order if c in df.columns]
    df = df[cols_order]

    # Sort by Final Accuracy (extract mean value for sorting)
    def extract_mean_for_sorting(x):
        try:
            return float(x.split(' ± ')[0])
        except:
            return 0.0

    if "Final Accuracy" in df.columns:
        df = df.sort_values(by=["Final Accuracy"], key=lambda x: x.apply(extract_mean_for_sorting), ascending=[False])

    print(df.to_string(index=False))

    out_csv = "adult_governance_results_mean_std_3_runs.csv"
    df.to_csv(out_csv, index=False)
    print(f"\n💾 Saved results to '{out_csv}'")

    # Also save raw results for reference
    raw_results = []
    for run_id, run_results in enumerate(all_runs_results):
        for scenario_result in run_results:
            scenario_result["Run"] = run_id + 1
            raw_results.append(scenario_result)

    raw_df = pd.DataFrame(raw_results)
    raw_out_csv = "adult_governance_results_raw_3_runs.csv"
    raw_df.to_csv(raw_out_csv, index=False)
    print(f"💾 Saved raw results to '{raw_out_csv}'")


if __name__ == "__main__":
    main()


⬆️ Please upload adult.csv ...



Saving adult.csv to adult.csv
Using uploaded file: adult.csv

🏃‍♂️ RUN 1/3
📊 Loading & preprocessing Adult dataset...
   Train: 21113, Test: 9049
   Features: 104

🔧 Creating Non-IID clients (Dirichlet α=0.5)
   Client 0: 6525 samples → {np.int64(0): np.int64(6338), np.int64(1): np.int64(187)}
   Client 1: 182 samples → {np.int64(0): np.int64(160), np.int64(1): np.int64(22)}
   Client 2: 2019 samples → {np.int64(0): np.int64(1992), np.int64(1): np.int64(27)}
   Client 3: 1381 samples → {np.int64(0): np.int64(1377), np.int64(1): np.int64(4)}
   Client 4: 1766 samples → {np.int64(0): np.int64(1483), np.int64(1): np.int64(283)}
   Client 5: 1538 samples → {np.int64(0): np.int64(367), np.int64(1): np.int64(1171)}
   Client 6: 543 samples → {np.int64(0): np.int64(43), np.int64(1): np.int64(500)}
   Client 7: 4459 samples → {np.int64(0): np.int64(2284), np.int64(1): np.int64(2175)}
   Client 8: 831 samples → {np.int64(0): np.int64(295), np.int64(1): np.int64(536)}
   Client 9: 1869 samples →

# **changing dirichlet parameter from 0.5 to 0.1 more non-IID distribution split**

In [2]:
# ============================================================
#  fEDERATED LEARNING — ADULT INCOME DATASET
#  Robust FL + Governance: FedAvg, FedProx, Median, Krum,
#  Power-of-Choice, Static PScore, Dynamic PScore, PScore-weighted
# ============================================================

import os
import time
import math
import random
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    recall_score,
    roc_auc_score,
)

import torch
import torch.nn as nn
import torch.optim as optim

# ============================================================
#  GLOBAL CONFIG
# ============================================================

RANDOM_STATE = 42
N_CLIENTS = 10
N_ROUNDS = 15
ALPHA_DIRICHLET = 0.1

BATCH_SIZE = 64
LOCAL_EPOCHS = 4
LR = 1e-3

DEVICE_POWER_KW = 0.1
CO2_PER_KWH = 0.4
MODEL_SIZE_MB = 1.0

# Robust FL / selection parameters
POC_TOP_K = 3              # Power-of-Choice
KRUM_F_BYZANTINE = 1       # Assumed number of Byzantine clients for Krum

# Static PScore threshold percentile (Gate 1)
GOVERNANCE_PERCENTILE = 30.0  # 30th percentile for basic gate

# Dynamic PScore-v3 hyperparams (soft down-weighting)
DYN_WARMUP_ROUNDS = 5
DYN_ALPHA_PROV = 0.7      # weight for governance vs sample-size
DYN_LOW_PENALTY = 0.5     # down-weight for low-provenance clients (less harsh than 0.3)

# Dynamic gate hyperparams (hard gate after warm-up)
GATE_WARMUP_ROUNDS = 5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
#  UTILS
# ============================================================

def set_global_seed(seed: int = RANDOM_STATE):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def estimate_energy_kwh(t_s: float) -> float:
    return DEVICE_POWER_KW * (t_s / 3600.0)


def estimate_co2_kg(kwh: float) -> float:
    return kwh * CO2_PER_KWH

# ============================================================
#  DATA LOADING + PREPROCESSING
# ============================================================

def load_adult_dataset(csv_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    print("📊 Loading & preprocessing Adult dataset...")

    df = pd.read_csv(csv_path, header=0, na_values=["?"])
    expected_cols = 15

    if df.shape[1] == expected_cols:
        df.columns = [
            "age", "workclass", "fnlwgt", "education", "education-num",
            "marital-status", "occupation", "relationship", "race", "sex",
            "capital-gain", "capital-loss", "hours-per-week",
            "native-country", "income"
        ]

    df = df.dropna().reset_index(drop=True)

    # Normalize income labels
    df["income"] = (
        df["income"]
        .astype(str)
        .str.lower()
        .str.strip()
        .str.replace(" ", "", regex=False)
    )

    # Remove header-like garbage rows if any
    mask_bad = df["income"].isin(["income", "income."])
    if mask_bad.any():
        print(f"   ⚠️ Removing {mask_bad.sum()} header-like rows")
        df = df[~mask_bad].reset_index(drop=True)

    mapping = {
        "<=50k": 0,
        ">50k": 1,
        "<=50k,": 0,
        ">50k,": 1,
    }
    y = df["income"].map(mapping)

    if y.isna().any():
        df["income"] = (
            df["income"]
            .str.replace("K", "k", regex=False)
            .str.replace(",", "", regex=False)
        )
        y = df["income"].map(mapping)
        if y.isna().any():
            print("   ⚠️ Fallback mapping on weird labels")
            y = df["income"].apply(lambda v: 0 if "<=50" in v else 1)

    y = y.astype(int).values

    num_cols = ["age", "fnlwgt", "education-num",
                "capital-gain", "capital-loss", "hours-per-week"]
    cat_cols = [c for c in df.columns if c not in num_cols + ["income"]]

    scaler = StandardScaler()
    X_num = scaler.fit_transform(df[num_cols].values)

    enc = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    X_cat = enc.fit_transform(df[cat_cols])

    X = np.hstack([X_num, X_cat]).astype(np.float32)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
    )

    print(f"   Train: {len(y_train)}, Test: {len(y_test)}")
    print(f"   Features: {X.shape[1]}\n")
    return X_train, X_test, y_train, y_test

# ============================================================
#  NON-IID DIRICHLET CLIENT SPLITTER
# ============================================================

def create_dirichlet_clients(
    X: np.ndarray, y: np.ndarray, n_clients: int, alpha: float
) -> List[Dict]:

    print(f"🔧 Creating Non-IID clients (Dirichlet α={alpha})")

    n_classes = len(np.unique(y))
    idx_by_class = [np.where(y == k)[0] for k in range(n_classes)]
    client_indices = [[] for _ in range(n_clients)]

    for k in range(n_classes):
        idx_k = idx_by_class[k]
        np.random.shuffle(idx_k)

        proportions = np.random.dirichlet(alpha * np.ones(n_clients))
        proportions = (proportions * len(idx_k)).astype(int)

        diff = len(idx_k) - proportions.sum()
        for i in range(abs(diff)):
            proportions[i % n_clients] += int(np.sign(diff))

        start = 0
        for cid in range(n_clients):
            end = min(start + proportions[cid], len(idx_k))
            client_indices[cid].extend(idx_k[start:end])
            start = end

    clients = []
    for cid in range(n_clients):
        idx = np.array(client_indices[cid])
        Xc, yc = X[idx], y[idx]
        dist = dict(zip(*np.unique(yc, return_counts=True)))
        print(f"   Client {cid}: {len(yc)} samples → {dist}")
        clients.append({"id": cid, "X": Xc, "y": yc})

    print("✅ Clients created.\n")
    return clients

# ============================================================
#  MODEL
# ============================================================

class AdultNet(nn.Module):
    def __init__(self, in_features: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        return self.net(x).squeeze(-1)


def init_global_model(in_features: int) -> nn.Module:
    return AdultNet(in_features).to(DEVICE)


def get_model_params(model: nn.Module) -> Dict[str, torch.Tensor]:
    return {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}


def set_model_params(model: nn.Module, params: Dict[str, torch.Tensor]):
    model.load_state_dict(params, strict=True)


def average_state_dicts(
    param_list: List[Dict[str, torch.Tensor]],
    weights: List[float],
) -> Dict[str, torch.Tensor]:
    weights = np.array(weights, dtype=float)
    weights = weights / (weights.sum() + 1e-12)

    avg = {}
    for k in param_list[0].keys():
        stacked = torch.stack([
            p[k].float() * float(w) for p, w in zip(param_list, weights)
        ])
        avg[k] = stacked.sum(dim=0)
    return avg

# ============================================================
#  ROBUST AGGREGATION HELPERS (MEDIAN, KRUM)
# ============================================================

def median_aggregate(param_list: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
    out = {}
    for k in param_list[0].keys():
        stacked = torch.stack([p[k].float() for p in param_list])  # [C, ...]
        median_vals = torch.median(stacked, dim=0).values
        out[k] = median_vals
    return out


def _param_distance(p1: Dict[str, torch.Tensor], p2: Dict[str, torch.Tensor]) -> float:
    dist = 0.0
    for k in p1.keys():
        diff = (p1[k].float() - p2[k].float()).view(-1)
        dist += torch.sum(diff * diff).item()
    return dist


def krum_aggregate(param_list: List[Dict[str, torch.Tensor]], f_byz: int) -> Tuple[Dict[str, torch.Tensor], int]:
    """
    Classical Krum:
      - For each client update i, compute sum of distances to its
        closest (m - f_byz - 2) other updates.
      - Choose update with minimal score.
    """
    m = len(param_list)
    if m <= 2 * f_byz + 2:
        # Not enough clients for Krum assumptions, fallback to FedAvg
        avg = average_state_dicts(param_list, [1.0] * m)
        return avg, -1

    dists = np.zeros((m, m), dtype=float)
    for i in range(m):
        for j in range(i + 1, m):
            d = _param_distance(param_list[i], param_list[j])
            dists[i, j] = dists[j, i] = d

    scores = []
    for i in range(m):
        # distances from i to others
        others = np.delete(dists[i], i)
        # sort and sum top m - f_byz - 2 closest
        nb_closest = m - f_byz - 2
        score_i = np.sum(np.partition(others, nb_closest)[:nb_closest])
        scores.append(score_i)

    winner = int(np.argmin(scores))
    return param_list[winner], winner

# ============================================================
#  LOCAL TRAINING (FEDAVG) + FEDPROX
# ============================================================

CRITERION = nn.BCEWithLogitsLoss()


def local_train(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
) -> Tuple[Dict[str, torch.Tensor], float, float]:

    """
    Standard local training used by FedAvg, Median, Krum, PoC, etc.
    """

    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    n_features = X.shape[1]
    local_model = AdultNet(n_features).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.float32)).float().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            loss = CRITERION(logits, yb)
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).cpu().numpy()

    acc = accuracy_score(y, preds)

    return get_model_params(local_model), train_time, acc


def local_train_fedprox(
    global_model: nn.Module,
    X: np.ndarray,
    y: np.ndarray,
    mu: float,
) -> Tuple[Dict[str, torch.Tensor], float, float]:
    """
    Local training with FedProx proximal term:
      loss_total = BCE + (mu/2) * ||w_local - w_global||^2
    """
    if X.shape[0] == 0:
        return get_model_params(global_model), 0.0, 0.0

    n_features = X.shape[1]
    local_model = AdultNet(n_features).to(DEVICE)
    local_model.load_state_dict(global_model.state_dict())

    global_params = {k: v.detach().clone() for k, v in global_model.state_dict().items()}

    optimizer = optim.Adam(local_model.parameters(), lr=LR)

    X_tensor = torch.from_numpy(X).float().to(DEVICE)
    y_tensor = torch.from_numpy(y.astype(np.float32)).float().to(DEVICE)

    n_samples = X_tensor.shape[0]
    indices = np.arange(n_samples)

    start = time.time()
    local_model.train()

    for _ in range(LOCAL_EPOCHS):
        np.random.shuffle(indices)
        for s in range(0, n_samples, BATCH_SIZE):
            e = min(s + BATCH_SIZE, n_samples)
            idx = indices[s:e]
            xb = X_tensor[idx]
            yb = y_tensor[idx]

            optimizer.zero_grad()
            logits = local_model(xb)
            base_loss = CRITERION(logits, yb)

            # Proximal term
            prox_term = 0.0
            for (name, param) in local_model.named_parameters():
                prox_term = prox_term + torch.sum((param - global_params[name].to(DEVICE))**2)

            loss = base_loss + (mu / 2.0) * prox_term
            loss.backward()
            optimizer.step()

    train_time = time.time() - start

    local_model.eval()
    with torch.no_grad():
        logits = local_model(X_tensor)
        probs = torch.sigmoid(logits)
        preds = (probs >= 0.5).cpu().numpy()

    acc = accuracy_score(y, preds)

    return get_model_params(local_model), train_time, acc

# ============================================================
#  GLOBAL EVALUATION
# ============================================================

def evaluate_global_model(
    model: nn.Module,
    X_test: np.ndarray,
    y_test: np.ndarray,
) -> Dict[str, float]:

    model.eval()
    with torch.no_grad():
        X_t = torch.from_numpy(X_test).float().to(DEVICE)
        logits = model(X_t)
        probs = torch.sigmoid(logits).cpu().numpy()
        preds = (probs >= 0.5).astype(int)

    acc = accuracy_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    rec = recall_score(y_test, preds)

    try:
        auc = roc_auc_score(y_test, probs)
    except ValueError:
        auc = float("nan")

    return {"accuracy": acc, "f1": f1, "recall": rec, "auc": auc}

# ============================================================
#  PSCORE UTILITIES
# ============================================================

def get_static_pscores(n_clients: int) -> List[float]:
    # Example PScore vector from your runs
    base = [
        4.05, 3.92, 3.94, 3.03, 3.28,
        2.83, 2.39, 1.58, 2.37, 1.51
    ]
    return base[:n_clients]


def compute_governance_threshold(pscores: List[float], percentile: float) -> float:
    arr = np.array(pscores, dtype=float)
    return float(np.percentile(arr, percentile))

def minmax_normalize_pscores(pscores: List[float]) -> np.ndarray:
    ps = np.array(pscores, dtype=float)
    p_min, p_max = ps.min(), ps.max()
    p_rng = p_max - p_min if p_max > p_min else 1.0
    return (ps - p_min) / p_rng

# ============================================================
#  BASELINE FEDAVG
# ============================================================

def run_fedavg(clients, X_test, y_test, pscores):
    print("🎯 FedAvg")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedAvg",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  FEDPROX
# ============================================================

def run_fedprox(clients, X_test, y_test, pscores, mu: float = 0.01):
    print("\n🎯 FedProx (mu={})".format(mu))

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train_fedprox(global_model, c["X"], c["y"], mu)
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "FedProx",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  MEDIAN
# ============================================================

def run_median(clients, X_test, y_test, pscores):
    print("\n🎯 Median")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg = median_aggregate(local_params)
        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Median",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }

# ============================================================
#  KRUM
# ============================================================

def run_krum(clients, X_test, y_test, pscores):
    print("\n🎯 Krum")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    chosen_counts = [0] * len(clients)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params = []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)

        agg, chosen = krum_aggregate(local_params, KRUM_F_BYZANTINE)
        if chosen >= 0:
            chosen_counts[chosen] += 1
            print(f"     🟢 Krum chose client ~{chosen}")
        else:
            print("     🟢 Fallback to FedAvg (not enough clients for Krum)")

        set_model_params(global_model, agg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "Krum",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": 1,
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),  # Krum doesn't use PScore
    }

# ============================================================
#  POWER-OF-CHOICE (TOP-K ACC)
# ============================================================

def run_power_of_choice(clients, X_test, y_test, pscores):
    print("\n🎯 Power-of-Choice (top-{} acc)".format(POC_TOP_K))

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    last_selected = []

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights, accs = [], [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t
            local_params.append(params)
            weights.append(len(c["y"]))
            accs.append(acc)

        accs = np.array(accs)
        top_idx = np.argsort(-accs)[:POC_TOP_K]
        last_selected = top_idx.tolist()
        print(f"     Selected clients: {last_selected}")

        sel_params = [local_params[i] for i in top_idx]
        sel_weights = [weights[i] for i in top_idx]

        avg = average_state_dicts(sel_params, sel_weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * POC_TOP_K * MODEL_SIZE_MB

    return {
        "Scenario": "Power-of-Choice",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(last_selected),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in last_selected])),
    }

# ============================================================
#  PSCORE-WEIGHTED FEDAVG (ALL CLIENTS, NO GATE)
# ============================================================

def run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores):
    """
    All clients always participate.
    Aggregation weight: |D_i| * f(P_i) with monotone PScore scaling.
    """
    print("\n🎯 PScore-Weighted FedAvg (all clients, no gate)")

    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)
    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)  # in [0,1]
    # map to e.g. [0.5, 1.5]
    f_ps = 0.5 + 1.0 * ps_norm

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")

        local_params, weights = [], []
        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * float(f_ps[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "PScoreWeightedFedAvgAll",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean(pscores)),
    }


# ============================================================
#  STATIC PSCORE (HARD GATE + PSCORE-WEIGHTED FEDAVG)
# ============================================================

def run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov: float):
    print("\n🎯 Static PScore Gate (hard gate + PScore-weighted FedAvg)")
    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)

    accepted_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    print(f"   Threshold={t_gov:.3f} → accepted: {accepted_ids}")

    active = [c for c in clients if c["id"] in accepted_ids]
    total_t = 0.0

    if len(active) == 0:
        print("   ⚠️ No clients above threshold; falling back to FedAvg over all clients.")
        return run_fedavg(clients, X_test, y_test, pscores)

    ps_norm = minmax_normalize_pscores(pscores)

    for r in range(1, N_ROUNDS + 1):
        print(f"  Round {r}")
        local_params, weights = [], []

        for c in active:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            eff_w = len(c["y"]) * (0.5 + 1.0 * ps_norm[cid])
            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(active) * MODEL_SIZE_MB

    return {
        "Scenario": "StaticPScoreGate",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(active),
        "Trained Clients": len(active),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in accepted_ids])),
    }

# ============================================================
#  DYNAMIC PSCORE-v3 (SOFT PENALTY, NO FILTERING)
# ============================================================

def run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov: float):
    """
    Warm-up rounds: vanilla FedAvg over all clients.
    After warm-up: all clients train, but aggregation weights reflect
    static PScore + governance penalty for low-provenance clients.
    """
    print("\n🎯 Dynamic PScore-v3 (warm-up + governance-weighted FedAvg)")
    n_features = clients[0]["X"].shape[1]
    global_model = init_global_model(n_features)

    total_t = 0.0

    ps_norm = minmax_normalize_pscores(pscores)

    high_ids = [i for i, s in enumerate(pscores) if s >= t_gov]
    low_ids = [i for i, s in enumerate(pscores) if s < t_gov]

    print(f"   Static threshold={t_gov:.3f}")
    print(f"   High-provenance clients: {high_ids}")
    print(f"   Low-provenance clients (penalized): {low_ids}")

    for r in range(1, N_ROUNDS + 1):
        mode = "warm-up" if r <= DYN_WARMUP_ROUNDS else "gov-weighted"
        print(f"  Round {r} [{mode}]")

        local_params, weights = [], []

        for c in clients:
            cid = c["id"]
            params, t, acc = local_train(global_model, c["X"], c["y"])
            print(f"     Client {cid} acc={acc:.3f}")
            total_t += t

            if r <= DYN_WARMUP_ROUNDS:
                eff_w = len(c["y"])
            else:
                norm_p = ps_norm[cid]
                if cid in low_ids:
                    norm_p *= DYN_LOW_PENALTY
                # Mix governance weight with sample-size
                g_factor = DYN_ALPHA_PROV * norm_p + (1.0 - DYN_ALPHA_PROV)
                eff_w = len(c["y"]) * max(g_factor, 1e-3)

            local_params.append(params)
            weights.append(eff_w)

        avg = average_state_dicts(local_params, weights)
        set_model_params(global_model, avg)
        m = evaluate_global_model(global_model, X_test, y_test)
        print(f"     🟢 Acc={m['accuracy']:.4f}")

    final = evaluate_global_model(global_model, X_test, y_test)
    energy = estimate_energy_kwh(total_t)
    co2 = estimate_co2_kg(energy)
    comm = N_ROUNDS * len(clients) * MODEL_SIZE_MB

    return {
        "Scenario": "DynamicPScoreV3",
        "Final Accuracy": final["accuracy"],
        "Final F1-Score": final["f1"],
        "Final Recall": final["recall"],
        "Final AUC-ROC": final["auc"],
        "Total Training Time (s)": total_t,
        "Total Energy (kWh)": energy,
        "Total CO2 (kg)": co2,
        "Total Communication (MB)": comm,
        "Selected Clients": len(clients),       # all contribute, but reweighted
        "Trained Clients": len(clients),
        "Avg PScore Selected": float(np.mean([pscores[i] for i in high_ids])),
    }

# ============================================================
#  MAIN PIPELINE WITH MULTIPLE RUNS
# ============================================================

def get_dataset_path():
    """Get dataset path - only prompt once"""
    try:
        from google.colab import files
        print("\n⬆️ Please upload adult.csv ...\n")
        uploaded = files.upload()
        if len(uploaded) == 0:
            raise ValueError("❌ No file uploaded.")
        fname = list(uploaded.keys())[0]
        print(f"Using uploaded file: {fname}")
        return fname
    except ImportError:
        if not os.path.exists("adult.csv"):
            raise FileNotFoundError(
                "❌ Not in Colab AND no local 'adult.csv' found.\n"
                "Please put adult.csv in your working directory."
            )
        print("Using local adult.csv")
        return "adult.csv"

def run_single_experiment(run_id: int, csv_path: str):
    """Run a single experiment and return results"""
    print(f"\n{'='*80}")
    print(f"🏃‍♂️ RUN {run_id + 1}/3")
    print(f"{'='*80}")

    set_global_seed(RANDOM_STATE + run_id)  # Different seed for each run

    # ----------------------------------------
    # 2) Load dataset
    # ----------------------------------------
    X_train, X_test, y_train, y_test = load_adult_dataset(csv_path)

    # ----------------------------------------
    # 3) Create Non-IID clients
    # ----------------------------------------
    clients = create_dirichlet_clients(
        X_train,
        y_train,
        n_clients=N_CLIENTS,
        alpha=ALPHA_DIRICHLET,
    )

    # ----------------------------------------
    # 4) PScore vector + governance threshold
    # ----------------------------------------
    pscores = get_static_pscores(N_CLIENTS)
    print("📊 Static PScore values:")
    for cid, s in enumerate(pscores):
        print(f"   Client {cid}: {s:.2f}")
    print()

    t_gov = compute_governance_threshold(pscores, GOVERNANCE_PERCENTILE)
    print(f"🔒 Static governance threshold from {GOVERNANCE_PERCENTILE:.0f}th percentile: {t_gov:.3f}\n")

    # ----------------------------------------
    # 5) Run baselines + governance variants
    # ----------------------------------------
    all_results = []

    # Baselines
    all_results.append(run_fedavg(clients, X_test, y_test, pscores))
    all_results.append(run_fedprox(clients, X_test, y_test, pscores, mu=0.01))
    all_results.append(run_median(clients, X_test, y_test, pscores))
    all_results.append(run_krum(clients, X_test, y_test, pscores))
    all_results.append(run_power_of_choice(clients, X_test, y_test, pscores))

    # Governance-aware variants
    all_results.append(run_pscore_weighted_fedavg_all(clients, X_test, y_test, pscores))
    all_results.append(run_static_pscore_gate(clients, X_test, y_test, pscores, t_gov))
    all_results.append(run_dynamic_pscore_v3(clients, X_test, y_test, pscores, t_gov))

    return all_results


def compute_statistics(all_runs_results):
    """Compute mean ± std for all metrics across runs"""
    # Get all unique scenario names from all runs
    all_scenarios = set()
    for run_results in all_runs_results:
        for scenario_result in run_results:
            all_scenarios.add(scenario_result["Scenario"])

    # Initialize storage for metrics
    metrics = {}
    for scenario in all_scenarios:
        metrics[scenario] = {
            "Final Accuracy": [],
            "Final F1-Score": [],
            "Final Recall": [],
            "Final AUC-ROC": [],
            "Total Training Time (s)": [],
            "Total Energy (kWh)": [],
            "Total CO2 (kg)": [],
            "Total Communication (MB)": [],
            "Selected Clients": [],
            "Trained Clients": [],
            "Avg PScore Selected": [],
        }

    # Collect data from all runs
    for run_results in all_runs_results:
        for scenario_result in run_results:
            scenario_name = scenario_result["Scenario"]
            for metric in metrics[scenario_name].keys():
                if metric in scenario_result:
                    metrics[scenario_name][metric].append(scenario_result[metric])

    # Compute statistics
    stats_results = []
    for scenario_name, scenario_metrics in metrics.items():
        stats_row = {"Scenario": scenario_name}

        for metric_name, values in scenario_metrics.items():
            if values:  # Only compute if we have data
                mean_val = np.mean(values)
                std_val = np.std(values)

                if metric_name in ["Final Accuracy", "Final F1-Score", "Final Recall", "Final AUC-ROC"]:
                    # For metrics where we want mean ± std format
                    stats_row[metric_name] = f"{mean_val:.4f} ± {std_val:.4f}"
                elif metric_name in ["Total Training Time (s)", "Total Energy (kWh)", "Total CO2 (kg)"]:
                    stats_row[metric_name] = f"{mean_val:.2f} ± {std_val:.2f}"
                elif metric_name in ["Total Communication (MB)", "Selected Clients", "Trained Clients"]:
                    stats_row[metric_name] = f"{mean_val:.1f} ± {std_val:.1f}"
                elif metric_name == "Avg PScore Selected":
                    stats_row[metric_name] = f"{mean_val:.2f} ± {std_val:.2f}"
                else:
                    stats_row[metric_name] = f"{mean_val:.4f} ± {std_val:.4f}"
            else:
                stats_row[metric_name] = "N/A"

        stats_results.append(stats_row)

    return stats_results


def main():
    # Get dataset path once
    csv_path = get_dataset_path()

    # Run experiment 3 times
    all_runs_results = []

    for run_id in range(3):
        run_results = run_single_experiment(run_id, csv_path)
        all_runs_results.append(run_results)

    # Compute statistics
    stats_results = compute_statistics(all_runs_results)

    # ----------------------------------------
    # Final summary with statistics
    # ----------------------------------------
    print("\n" + "=" * 120)
    print("📊 FINAL RESULTS — Adult Dataset (Governance + Robust FL) - Mean ± Std over 3 runs")
    print("=" * 120)

    df = pd.DataFrame(stats_results)

    cols_order = [
        "Scenario",
        "Final Accuracy",
        "Final F1-Score",
        "Final Recall",
        "Final AUC-ROC",
        "Total Training Time (s)",
        "Total Energy (kWh)",
        "Total CO2 (kg)",
        "Total Communication (MB)",
        "Selected Clients",
        "Trained Clients",
        "Avg PScore Selected",
    ]
    cols_order = [c for c in cols_order if c in df.columns]
    df = df[cols_order]

    # Sort by Final Accuracy (extract mean value for sorting)
    def extract_mean_for_sorting(x):
        try:
            return float(x.split(' ± ')[0])
        except:
            return 0.0

    if "Final Accuracy" in df.columns:
        df = df.sort_values(by=["Final Accuracy"], key=lambda x: x.apply(extract_mean_for_sorting), ascending=[False])

    print(df.to_string(index=False))

    out_csv = "adult_governance_results_mean_std_3_runs.csv"
    df.to_csv(out_csv, index=False)
    print(f"\n💾 Saved results to '{out_csv}'")

    # Also save raw results for reference
    raw_results = []
    for run_id, run_results in enumerate(all_runs_results):
        for scenario_result in run_results:
            scenario_result["Run"] = run_id + 1
            raw_results.append(scenario_result)

    raw_df = pd.DataFrame(raw_results)
    raw_out_csv = "adult_governance_results_raw_3_runs.csv"
    raw_df.to_csv(raw_out_csv, index=False)
    print(f"💾 Saved raw results to '{raw_out_csv}'")


if __name__ == "__main__":
    main()


⬆️ Please upload adult.csv ...



Saving adult.csv to adult (1).csv
Using uploaded file: adult (1).csv

🏃‍♂️ RUN 1/3
📊 Loading & preprocessing Adult dataset...
   Train: 21113, Test: 9049
   Features: 104

🔧 Creating Non-IID clients (Dirichlet α=0.1)
   Client 0: 11126 samples → {np.int64(0): np.int64(11125), np.int64(1): np.int64(1)}
   Client 1: 11 samples → {np.int64(0): np.int64(1), np.int64(1): np.int64(10)}
   Client 2: 1313 samples → {np.int64(0): np.int64(1312), np.int64(1): np.int64(1)}
   Client 3: 444 samples → {np.int64(0): np.int64(443), np.int64(1): np.int64(1)}
   Client 4: 562 samples → {np.int64(0): np.int64(562)}
   Client 5: 571 samples → {np.int64(0): np.int64(567), np.int64(1): np.int64(4)}
   Client 6: 48 samples → {np.int64(0): np.int64(1), np.int64(1): np.int64(47)}
   Client 7: 1134 samples → {np.int64(1): np.int64(1134)}
   Client 8: 67 samples → {np.int64(1): np.int64(67)}
   Client 9: 5837 samples → {np.int64(0): np.int64(1846), np.int64(1): np.int64(3991)}
✅ Clients created.

📊 Static PScor

# **END OF EXPERIMENTS**